In [1]:
import pandas as pd
import geopandas as gpd
import requests
import json
from bs4 import BeautifulSoup


In [ ]:
# Yelp Search Results
r = requests.get('https://www.yelp.com/search?find_desc=bars&find_loc=Los+Angeles%2C+CA+90049&ns=1')
r.ok

In [ ]:
#r.text

In [ ]:
soup = BeautifulSoup(r.text)
print(soup.prettify())

In [ ]:
# YELP URL of One page (could extract from businessurl)
r = requests.get('https://www.yelp.com/biz/the-wellesbourne-los-angeles?osq=bars')

In [ ]:
soup = BeautifulSoup(r.text)
print(soup.prettify())

In [ ]:
soup.find_all("application/ld+json")

In [7]:
# Foursquare API
# API has tips for each venue, can pull up to 
import requests

fsq_id = '5a187743ccad6b307315e6fe'
url = "https://api.foursquare.com/v3/places/{}/tips".format(fsq_id)

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

Header overflow
